## Initializing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
plt.style.use("seaborn-dark")

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from imblearn.metrics import classification_report_imbalanced

import statsmodels.api as sm
import statsmodels.formula.api as smf

import xgboost as xgb

## Importing Data

In [2]:
df = pd.read_csv('coding_round_data.csv')

### Fixing import issues

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [4]:
for col in ['OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Month']:
    df[col] = df[col].astype('category')

for col in ['Weekend', 'Revenue']:
    df[col] = df[col].astype('int')

## Splitting Data

Using a stratified split as the data is very imbalanced

In [5]:
df_train, df_test = train_test_split(df, stratify=df['Revenue'], test_size=0.20, random_state=42)

## Column Manipulation

In [6]:
nominal_features = ['Month','OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType','Weekend']
nominal_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("nominal", nominal_transformer, nominal_features),
    ],
    sparse_threshold=0,
    remainder='passthrough'
)

preprocessor.fit(df_train)

df_train = pd.DataFrame(preprocessor.transform(df_train), columns=preprocessor.get_feature_names_out())
df_test = pd.DataFrame(preprocessor.transform(df_test), columns=preprocessor.get_feature_names_out())

## Modelling

There are 2 main approaches that I will be taking for modelling. Their results will be compared on the df_test set to decide a final approach through the F1-score

### Approach 1 - Logistic Regression

In [7]:
formula = "remainder__Revenue ~ " + " + ".join(df_train.columns.drop('remainder__Revenue'))

LR_model = smf.logit(formula=formula, data=df_train).fit_regularized(method='l1')
LR_model.summary()

/home/kavish/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/kavish/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.28314604909132673
            Iterations: 466
            Function evaluations: 486
            Gradient evaluations: 466


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     remainder__Revenue   No. Observations:                 9864
Model:                          Logit   Df Residuals:                     9788
Method:                           MLE   Df Model:                           75
Date:                Sun, 03 Apr 2022   Pseudo R-squ.:                  0.3427
Time:                        17:40:56   Log-Likelihood:                -2793.0
converged:                       True   LL-Null:                       -4249.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                 -2.1828      1e+06  -2.18e-06      1.000   -1.96e+06    1.96e+06
nominal__Month_Aug                         0.2679        nan        nan        nan         nan         nan
nominal__Month_Dec                        -0.4336        nan        nan        nan         nan         nan
nominal__Month_Feb                        -1.7480        nan        nan        nan         nan         nan
nominal__Month_Jul                         0.1536        nan        nan        nan         nan         nan
nominal__Month_June                       -0.0799        nan        nan        nan         nan         nan
nominal__Month_Mar                        -0.3999        nan        nan        nan         nan         nan
nominal__Month_May                        -0.3252        nan        nan        nan         nan         nan
nominal__Month_Nov                         0.5935        nan        nan        nan         nan         nan
nominal__Month_Oct                         0.1700        nan        nan        nan         nan         nan
nominal__Month_Sep                         0.3057        nan        nan        nan         nan         nan
nominal__OperatingSystems_1               -0.1742        nan        nan        nan         nan         nan
nominal__OperatingSystems_2               -0.0050        nan        nan        nan         nan         nan
nominal__OperatingSystems_3               -0.2459        nan        nan        nan         nan         nan
nominal__OperatingSystems_4               -0.2310        nan        nan        nan         nan         nan
nominal__OperatingSystems_5               -0.1088   9.49e+07  -1.15e-09      1.000   -1.86e+08    1.86e+08
nominal__OperatingSystems_6               -1.1027        nan        nan        nan         nan         nan
nominal__OperatingSystems_7                1.4606        nan        nan        nan         nan         nan
nominal__OperatingSystems_8               -0.0651        nan        nan        nan         nan         nan
nominal__Browser_1                        -0.1679   9.36e+05  -1.79e-07      1.000   -1.83e+06    1.83e+06
nominal__Browser_2                        -0.2979    9.3e+05   -3.2e-07      1.000   -1.82e+06    1.82e+06
nominal__Browser_3                        -1.0189   9.12e+05  -1.12e-06      1.000   -1.79e+06    1.79e+06
nominal__Browser_4                        -0.2359   9.18e+05  -2.57e-07      1.000    -1.8e+06     1.8e+06
nominal__Browser_5                        -0.1171   8.88e+05  -1.32e-07      1.000   -1.74e+06    1.74e+06
nominal__Browser_6                        -0.5060      9e+05  -5.62e-07      1.000   -1.76e+06    1.76e+06
nominal__Browser_7                        -1.4221   9.18e+05  -1.55e-06      1.000    -1.8e+06     1.8e+06
nominal__Browser_8                        -0.6024   8.94e+05  -6.74e-07      1.000   -1.75e+06    1.75e+06
nominal__Browser_9             

As the classes are not balanced, the intercept/threshold needs to be scaled accordingly.

In [8]:
weights = df_train.remainder__Revenue.value_counts()

threshold = weights[1]/weights[0]

LR_test_preds_raw = LR_model.predict(exog=df_test.drop(columns=['remainder__Revenue']))
LR_test_preds = (LR_test_preds_raw > threshold).astype('int')

print(classification_report_imbalanced(df_test['remainder__Revenue'], LR_test_preds))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.94      0.87      0.71      0.91      0.79      0.63      2084
        1.0       0.50      0.71      0.87      0.59      0.79      0.61       382

avg / total       0.88      0.85      0.74      0.86      0.79      0.63      2466



### Approach 2 - Boosted Decision Trees

Implemented using XGBoost

In [9]:
X_train = df_train.drop(columns='remainder__Revenue')
y_train = df_train[['remainder__Revenue']]

X_test = df_test.drop(columns='remainder__Revenue')
y_test = df_test[['remainder__Revenue']]

In [10]:
sample_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)  # To help tackle imbalanced classes

xgb_model = xgb.XGBClassifier().fit(X_train, y_train, sample_weight=sample_weights)

/home/kavish/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/kavish/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kavish/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d

[17:40:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:
xgb_test_preds = xgb_model.predict(X_test)

print(classification_report_imbalanced(y_test, xgb_test_preds))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.94      0.90      0.71      0.92      0.80      0.65      2084
        1.0       0.58      0.71      0.90      0.64      0.80      0.63       382

avg / total       0.89      0.87      0.74      0.88      0.80      0.65      2466



## Results

The results from both approaches perform almost equally well on the test set with the logistic regression having an F1 score of 0.86 to the 0.88 for XGBoost